<a href="https://colab.research.google.com/github/benjaminnigjeh/DIAMS/blob/main/hyperParameterOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Keras_Tuner
keras_tuner is a library that will be used for optimization of hyperparameters. The library should be installed running pip from the shell.

In [ ]:
!pip install keras_tuner

# Import libraries and dependencies
google colab has already pre-installed versions of numpy, pandas, tensorflow, and sklearn.

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import KFold
from keras_tuner.tuners import RandomSearch
from google.colab import files
from keras.layers import Dropout

# Import training and test datasets
the datasets should be downloaded to a local folder from GitHub repository, and uploaded to colab running environment

In [ ]:
trainingDataset = files.upload_file('trainingDataset')

# Training Dataset wrangling

the sample labels are removed and numerical data transformed to numpy array

In [4]:
file_path = '/content/trainingDataset'
df = pd.read_csv(file_path)
Y = df.pop("target")
X = np.array(df)
y = np.array(Y)

# Helper functions to execute cycles of optimization

A model class is built wuth two dense layers which node numbers will be optimized

In [5]:
def build_model(hp):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Dense(hp.Int("input_units_1", min_value =100, max_value =200, step =5), activation=tf.nn.relu))
    model.add(Dropout(0.2))
    model.add(tf.keras.layers.Dense(hp.Int("input_units_2", min_value =100, max_value =300, step =5), activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')
    return model

tuner = RandomSearch(
    build_model,
    objective= "val_accuracy",
    max_trials= 100,
    executions_per_trial=1,
    directory= "logDir"
    )
kf = KFold(n_splits=4, shuffle=True, random_state=100)


# Optimization execution

a "for" cycle is executed to run the random search function of keras_tuner

In [ ]:
for train_index, validation_index in kf.split(X):
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y[train_index], y[validation_index]
    tuner.search(x=X_train,
                 y=y_train,
                 validation_data=(X_validation, y_validation))